## KB Fuzzy Matching

In [218]:
import pandas as pd

In [63]:
import Levenshtein as lev

In [58]:
import rapidfuzz

In [59]:
### Match based on 

In [78]:
kb_entities=pd.read_csv('kb_entities.csv',names=['id','entity','desc'])

In [61]:
rapidfuzz.fuzz.ratio('Boris Johnson', 'Bojo')

35.29411764705882

In [70]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Bojo')

66.66666666666667

In [156]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Boris Kevin Jonshon')

69.56521739130434

In [157]:
rapidfuzz.fuzz.token_set_ratio('Boris Johnson', 'Boris Kevin Jonshon')

75.0

In [178]:
rapidfuzz.fuzz.token_set_ratio('Boris K Johnson','Boris Johnson')

76.47058823529412

In [159]:
rapidfuzz.fuzz.WRatio('Boris Johnson', 'Boris Kevin Jonshon')

75.0

In [160]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Bojo')

66.66666666666667

In [161]:
rapidfuzz.fuzz.token_set_ratio('Boris Johnson', 'Bojo')

47.05882352941177

In [162]:
rapidfuzz.fuzz.WRatio('Boris Johnson', 'Bojo')

60.00000000000001

In [68]:
lev.distance('Boris Johnson', 'Bojo')

10

In [198]:
rapidfuzz.fuzz.token_set_ratio('Borris Johnson', 'Dr. Boris K. D. Johnson PhD')

68.42105263157895

In [186]:
rapidfuzz.fuzz.ratio('boris johnson', 'boris')

55.55555555555556

In [72]:
rapidfuzz.fuzz.partial_ratio('Boris Johnson', 'Boris')

100.0

In [69]:
lev.distance('Boris Johnson', 'Boris')

8

In [90]:
kb_entities[['id','entity']]

,id,entity
0,41210,John Burns
1,52782,Independent
2,37933,Sonia Sotomayor
3,36983,Samuel Alito
4,15108,Donald Trump
...,...,...
6751,243486,David Gordon
6752,407940,David Gordon
6753,412578,David Gordon
6754,41748,Palm


In [205]:
reference='Adam Smith'
distances={}
for entity_id in names.keys():
    distance=rapidfuzz.fuzz.token_set_ratio(reference,names[entity_id])
    distances[entity_id]=distance

In [206]:
K = 60
candidate_ids={key : val for key, val in distances.items() if val > K}
candidate_ids=list(candidate_ids.keys())

In [207]:
candidate_ids[0]

'53881'

In [208]:
kb_entities['id']=kb_entities['id'].astype(str)

In [209]:
candidate_df=kb_entities[kb_entities['id'].isin(candidate_ids)]

In [210]:
candidate_df.to_csv('candidates.csv')

In [127]:
### Match based on embedding similarity

In [174]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [175]:
names = dict()
descriptions = dict()
descriptions_vec=dict()

for row in candidate_df.iterrows():
    qid = row[1]['id']
    name = row[1]['entity']
    desc = row[1]['desc']
    names[qid] = name
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    descriptions[qid] = desc
    descriptions_vec[qid] = desc_enc

In [176]:
candidate_df[candidate_df['entity']=='Adam Smith']

,id,entity,desc
3415,13596,Adam Smith,US Representative from Washington (1997-presen...
3416,129552,Adam Smith,Moral philosopher and political economist
3417,256328,Adam Smith,San Diego realtor
3418,269916,Adam Smith,First National Bank of Coffee County
3419,379819,Adam Smith,Founder and CEO of Geoplex Australia


In [152]:
vector=descriptions_vec['13596']
distances={}
for entity_id in descriptions_vec.keys():
    vector_b=descriptions_vec[entity_id]
    distance=dot(vector, vector_b)/(norm(vector)*norm(vector_b))
    distances[entity_id]=distance

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in float_scalars
  """


In [153]:
res = dict(sorted(distances.items(), key = itemgetter(1), reverse = True)[:5])

In [154]:
res

{'13596': 0.99999994,
 '13573': 0.9508932,
 '13861': 0.9436378,
 '115495': 0.88238436,
 '200612': 0.88210636}

In [155]:
candidate_df[candidate_df['id'].isin(res.keys())]

,id,entity,desc
1104,13573,Adam Schiff,US Representative from California (2001-presen...
1600,115495,Eric Adams,\n Mayor of NYC; former Brooklyn Boroug...
1607,200612,Tim Smith,"President & CEO, Skidmore Studio Tim Smith, wh..."
1702,13861,Robert Smith,former US Representative and Senator from New ...
3415,13596,Adam Smith,US Representative from Washington (1997-presen...


In [48]:
import csv
from pathlib import Path

def load_entities():
    entities_loc = Path.cwd()/"kb_entities.csv"  # distributed alongside this notebook

    names = dict()
    descriptions = dict()
    descriptions_vec=dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            qid = row[0]
            name = row[1]
            desc = row[2]
            names[qid] = name
            desc_doc = nlp(desc)
            desc_enc = desc_doc.vector
            descriptions[qid] = desc
            descriptions_vec[qid] = desc_enc
    return names, descriptions, descriptions_vec

In [49]:
names, descriptions, descriptions_vec =load_entities()

In [ ]:
from numpy import dot
from numpy.linalg import norm

vector=descriptions['41210']
distances={}
for entity_id in descriptions.keys():
    vector_b=descriptions[entity_id]
    distance=dot(vector, vector_b)/(norm(vector)*norm(vector_b))
    distances[entity_id]=distance

In [36]:
from operator import itemgetter
res = dict(sorted(distances.items(), key = itemgetter(1), reverse = True)[:5])